In [1]:
pip install -U sacremoses sacrebleu evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from kaggle_secrets import UserSecretsClient
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import huggingface_hub
import numpy as np
import torch
import evaluate
import re

2024-06-11 17:43:21.021000: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 17:43:21.021101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 17:43:21.154542: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
num_workers = !cat /proc/cpuinfo | grep processor | wc -l
num_workers = int(num_workers[-1])

if torch.cuda.is_available():
    for gpu in range(torch.cuda.device_count()):
        print(f'GPU-{gpu}: {torch.cuda.get_device_name(gpu)}\nCPU: {num_workers}')
    device = torch.device('cuda:0')
else:
    print(f'GPU: None\nCPU: {num_workers}')
    device = torch.device('cpu')
    
print(f'Device: {device}')

GPU-0: Tesla P100-PCIE-16GB
CPU: 4
Device: cuda:0


In [4]:
user_secrets = UserSecretsClient()
huggingface_hub.login(token=user_secrets.get_secret("HF_TOKEN"))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
opus_dataset = load_dataset('Helsinki-NLP/opus-100', 'en-ru')
opus_dataset

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

In [6]:
opus_dataset['train'][2]

{'translation': {'en': 'The schedule below is tentative; up-to-date information can be obtained at www.un.org/News/ossg/conf.htm.',
  'ru': 'Приводимое ниже расписание является предварительным; с самой последней информацией можно ознакомиться в Интернете по адресу www.un.org/News/ossg/conf.htm.'}}

In [7]:
model_checkpoint = "Helsinki-NLP/opus-mt-uk-ru"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

In [8]:
regex = re.compile('[АаЕеЁёИиЙйОоУуЫыЭэЮюЯя]')
print(regex.sub('', 'Широкая электрификация южных губерний даст мощный толчок подъёму сельского хозяйства!'))
print(regex.sub('', 'Съешь еще этих мягких французских булок, да выпей чаю'))

Шрк лктрфкц жнх гбрн дст мщн тлчк пдъм сльскг хзств!
Съшь щ тх мгкх фрнцзскх блк, д вп ч


In [9]:
max_length = 128

def preprocess_function(examples):
    inputs = [regex.sub('', ex['ru']) for ex in examples['translation']]
    targets = [ex['ru'] for ex in examples['translation']]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs


tokenized_datasets = opus_dataset.map(preprocess_function, batched=True, remove_columns=opus_dataset["train"].column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

pytorch_model.bin:   0%|          | 0.00/297M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [11]:
metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [12]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-to-restore-ru-vowels",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    report_to='none',
)

In [13]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 4.1496710777282715,
 'eval_bleu': 13.327605052988618,
 'eval_runtime': 232.6987,
 'eval_samples_per_second': 8.595,
 'eval_steps_per_second': 0.138}

In [15]:
trainer.train()

Step,Training Loss
500,2.953300
1000,2.323800
1500,2.033900
2000,1.822100
2500,1.673400
3000,1.541600
3500,1.458400
4000,1.383300
4500,1.318400
5000,1.266500


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[57776]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[57776]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=93750, training_loss=0.6804891735839844, metrics={'train_runtime': 25934.8018, 'train_samples_per_second': 115.675, 'train_steps_per_second': 3.615, 'total_flos': 9.064012611531571e+16, 'train_loss': 0.6804891735839844, 'epoch': 3.0})

In [16]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 0.4423465132713318,
 'eval_bleu': 72.17871550607528,
 'eval_runtime': 196.7859,
 'eval_samples_per_second': 10.163,
 'eval_steps_per_second': 0.163,
 'epoch': 3.0}

In [17]:
trainer.push_to_hub(tags="translation", commit_message="initial")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[57776]], 'forced_eos_token_id': 0}


CommitInfo(commit_url='https://huggingface.co/avfawkes/marian-finetuned-to-restore-ru-vowels/commit/31432c565e676f39f4ddc0b081086f63ec5bfc71', commit_message='initial', commit_description='', oid='31432c565e676f39f4ddc0b081086f63ec5bfc71', pr_url=None, pr_revision=None, pr_num=None)